# Projeto 1 - Ciência dos Dados

Nome: João Gabriel Valentim Rocha

Nome: Enzo Dadier Lacks Zamberlan

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

In [76]:
!pip install seaborn
!pip install emoji
!pip install pysinonimos
!pip install nltk
!pip install sklearn

  Created wheel for emoji: filename=emoji-1.5.0-py3-none-any.whl size=187447 sha256=27aa3dbc73cb8d6664c0b6f79daa5b81c789451d7496485d1ca0db2f7db91bbc
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\e9\d5\46\33c9101a710eb267fbaf8572c96fce5b12702e62bb5fa40592
Successfully built emoji
  Created wheel for pysinonimos: filename=pysinonimos-0.1.1-py3-none-any.whl size=2458 sha256=aea325f593ed2b893c015dcfc5a2445575e1057d512f015876836ee199513126
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\52\3b\eb\f40f66159b8c2a9b9b25c4e1f91ff7f566b7f2305268adf062
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=12552 sha256=51d9858864eb605c304b359b55d897e22242f36e2ecd7c7dd08434a2da29b6bb
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\d1\d7\61\11b5b370ee487d38b5408ecb7e0257db9107fa622412cbe2ff
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7770 sha256=100548b250ffa97a0ec2a3a995a3db5

In [121]:
!pip install unidecode

___
Carregando algumas bibliotecas:

In [79]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [127]:
import re
import string
import emoji
import nltk
import pysinonimos.sinonimos as sinom
from emoji import UNICODE_EMOJI
import unidecode

In [81]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gabri\Desktop\Matérias 2 semestre\cdados\P1\Classificador_automatico_de_sentimentos


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [82]:
filename = 'assets/loki.xlsx'

In [141]:
train = pd.read_excel(filename)
train

,Treinamento,Relevancia
0,mood de hj: bolsominion veio falar com loki:\n...,0
1,sdds do loki,1
2,"considerações finais de loki: parabéns marvel,...",1
3,@_namizinhaa @yaluv_ @_girlrainbow versão femi...,0
4,o loki merece todo o amor do mundo ? \n \n si...,1
...,...,...
295,o loki tbm ? percebi que nao sei mais nada sob...,1
296,tô desde as 9 fazendo atividade e o loki destr...,0
297,@torystyliinson @mobiusdaavt pede pra ele te m...,0
298,"bolsonaro tá desesperado. discurso inflado, go...",0


In [215]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test

,Teste,Relevancia
0,comecei loki e logo no primeiro episódio ele s...,1
1,@thylaspirk exatamente \nnão dá cara loki mere...,1
2,pq a marvel falou que loki é irmão se thor sen...,0
3,vou fazer com o loki 🥰🤣 https://t.co/qry6pytktj,1
4,@stawocon loki eu ainda vi dnv ksksksksksksksk...,1
...,...,...
195,"vou fazer uma xícara temática pra mim, mas tô ...",0
196,o primeiro dente de leite de loki caiu https:/...,0
197,@mobiusdaavt @loki_l4ufeys0n eu vou te bloquear,0
198,queria ter juntado uma semana de merda de loki...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O classificador automático de sentimentos montado tem como critério utilizado a relevância de tweets que remetam a alguma reação ou laço sentimental em relação à série, sendo os mesmos positivos ou não (elogiando algum episódio, comentando valores da série, ou ainda, criticando os mesmos). Neste sentido, postagens que dizem respeito unicamente a aspectos nesse tocante, elogiando ou comentando sobre os atores da obra, como também tweets com vagas menções sobre a série, foram considerados como irrelevantes.

### Procedimentos e configurações iniciais

Para continuarmos, é necessário definir algumas funções básicas que vão nos ajudar:

1) Primeiro, vamos fazer a limpeza dos tweets. Retirando simbolos e pontuações que não serão contabilizadas para nossa análise

2)

In [115]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [176]:
# Passo 0 - Definição de transcrição de emojis
def separa_emoji(tweet):
    '''
    Essa função separa os emojis juntos, além de transcreve-los
    para suas respectivas strings descritivas, optimizando um pouco o processamento.
    Nota-se que alguns emojis não usados não estão no Unicode da língua portuguesa,
    adotamos assim o de língua inglesa como alternativa de transcrição.
    '''
    modified=' '.join(emoji.get_emoji_regexp().split(tweet))
    modified=modified.split()
    for i,emoji1 in enumerate(modified):
        if emoji1 in UNICODE_EMOJI['pt']:
            modified[i]=UNICODE_EMOJI['pt'][emoji1].replace(':','')
        elif emoji1 in UNICODE_EMOJI['en']:
            modified[i]=UNICODE_EMOJI['en'][emoji1].replace(':','')
        else:
            continue
    modified=' '.join(modified)
        
    return modified

In [272]:
# Passo 1
def limpa_frase(frase):
    # Primeiro, poe todas as palavras com letras minúsculas
    aux = frase.lower()
    
    # Segundo, remove # and @
    aux = re.sub("@[A-Za-z0-9_]+","", aux)
    aux = re.sub("#[A-Za-z0-9_]+","", aux)
    
    # Terceiro, remove links
    aux = re.sub(r"http\S+", "", aux)
    aux = re.sub(r"www.\S+", "", aux)
    
    # Quarto, remove pontuação
    aux = re.sub('[()!?]', ' ', aux)
    aux = re.sub('\[.*?\]',' ', aux)
    
    #
    aux = separa_emoji(aux)
    
    aux = aux.replace("_", "")
    
    # Quinto, remove acentos
    aux = unidecode.unidecode(aux)
    
    # Sexto, remove não alfa-numericos
    # ---------------rever--------------
    aux = re.sub("[^a-z0-9]"," ", aux)
    return aux

In [227]:
# Passo 2
def tokenize(frase):
    return frase.split()

In [193]:
# Passo 3 
nltk.download('stopwords')
prep = nltk.corpus.stopwords.words('portuguese')
prep.append('')
prep = prep + ['n', 's']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [221]:
# Passo 4
def no_stop_words(token):
    clear = []
    for element in token:
        if element not in prep:
            clear.append(element)
    return clear

In [222]:
def list_to_string(lista):
    return ' '.join(lista)

In [254]:
def limpa_tweet(tweet):
    return list_to_string(no_stop_words(tokenize(limpa_frase(tweet))))

In [251]:
def frases_to_series(frases):
    pass

In [267]:
frase_1 = test['Teste'][3]

In [273]:
print(no_stop_words(tokenize(limpa_frase(frase_1))))
frase_2 = list_to_string(no_stop_words(tokenize(limpa_frase(frase_1))))
print(frase_2)

['vou', 'fazer', 'loki', 'rostosorridentecom3coracoes', 'rolandonochaoderir']
vou fazer loki rostosorridentecom3coracoes rolandonochaoderir


In [274]:
print(separa_emoji(frase_2))

vou fazer loki rostosorridentecom3coracoes rolandonochaoderir


In [287]:
print(limpa_tweet(frase_1))

vou fazer loki rostosorridentecom3coracoes rolandonochaoderir


In [271]:
frase_3 = train['Treinamento'][2]
print(no_stop_words(tokenize(limpa_frase(frase_3))))

['consideracoes', 'finais', 'loki', 'parabens', 'marvel', 'chorei', 'igual', 'vagabunda', 'shipp', 'q', 'claramente', 'nao', 'endgame', 'matei', 'resto', 'neuronios', 'mt', 'bom', 'nota', '10']


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [284]:
#Transformando palavras em variáveis categóricas:
train['Treinamento'] = train['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [285]:
#Aplicando funções de limpeza e certificação no dataframe de treinamento da base de dados:
train['Clean']=train['Treinamento'].apply(limpa_tweet)
test['Clean']=test['Teste'].apply(limpa_tweet)

In [359]:
train_rel = train.loc[train['Relevancia'] == 1, :]
train_irrel= train.loc[train['Relevancia'] == 0, :]

palavras_rel = []
palavras_irrel = []
for i in range(len(train['Treinamento'])):
    if train['Relevancia'][i] == 1:
        tweet = train['Treinamento'][i]
        tweet = tokenize(limpa_tweet(tweet))
        palavras_rel += tweet
    else:
        tweet = train['Treinamento'][i]
        tweet = tokenize(limpa_tweet(tweet))
        palavras_irrel += tweet
        
palavras_rel = pd.Series(palavras_rel)
palavras_irrel = pd.Series(palavras_irrel)
aux = list(palavras_irrel) + list(palavras_rel)
palavras_totais = pd.Series(aux)

In [360]:
# Frequencias relativas
freq_rel_relevantes = palavras_rel.value_counts(True)
freq_rel_irrelevantes = palavras_irrel.value_counts(True)
freq_rel_total = palavras_totais.value_counts(True)
freq_rel_relevantes

loki                      0.109391
rostochorandoaosberros    0.019608
nao                       0.019608
thor                      0.015480
serie                     0.013416
                            ...   
riverdale                 0.001032
crime                     0.001032
terca                     0.001032
d                         0.001032
enganar                   0.001032
Length: 547, dtype: float64

In [362]:
'dou' in freq_rel_irrelevantes

False

In [363]:
# Lista com as palavras relevantes e irrelevantes
lista_rel = palavras_rel.to_list()
lista_irrel = palavras_irrel.to_list()

# Todas as palavras possiveis
todas_as_palavras = list(set(lista_rel + lista_irrel))

#### Probabilidades 


In [366]:
# Note que P_R (probabilidade de ser relevante) e P_Rc (probabilidade de ser irrelevante) são complementares!
P_R = len(palavras_rel) / (len(palavras_rel) + len(palavras_irrel))
P_Rc = 1 - P_R
'dou' in lista_rel

True

In [519]:
test

,Teste,Relevancia,Clean
0,comecei loki e logo no primeiro episódio ele s...,1,comecei loki logo primeiro episodio camisa obg...
1,@thylaspirk exatamente \nnão dá cara loki mere...,1,exatamente nao cara loki merecia
2,pq a marvel falou que loki é irmão se thor sen...,0,pq marvel falou loki irmao thor sendo q irmao ...
3,vou fazer com o loki 🥰🤣 https://t.co/qry6pytktj,1,vou fazer loki rostosorridentecom3coracoes rol...
4,@stawocon loki eu ainda vi dnv ksksksksksksksk...,1,loki ainda vi dnv ksksksksksksksksk agora what...
...,...,...,...
195,"vou fazer uma xícara temática pra mim, mas tô ...",0,vou fazer xicara tematica pra mim to duvida fa...
196,o primeiro dente de leite de loki caiu https:/...,0,primeiro dente leite loki caiu
197,@mobiusdaavt @loki_l4ufeys0n eu vou te bloquear,0,vou bloquear
198,queria ter juntado uma semana de merda de loki...,0,queria ter juntado semana merda loki pra jogar...


In [521]:
def probabilidadeCD (frase, arg):
    if arg == 'R':
        probFraseDadoR = 1
        for palavra in frase:
            try:
                probFraseDadoR *= freq_rel_relevantes[palavra]
            except:
                try:
                    probFraseDadoR *= freq_rel_total[palavra]
                except:
                    continue
        return probFraseDadoR
    if arg == 'I':
        probFraseDadoI = 1
        for palavra in frase:
            try:
                probFraseDadoI *= freq_rel_irrelevantes[palavra]
            except:
                try:
                    probFraseDadoI *= freq_rel_total[palavra]
                except:
                    continue
        return probFraseDadoI
i = 160
frase_1 = list(tokenize(train['Clean'][i]))
frase_1
probFraseDadoR, probFraseDadoI = probabilidadeCD(frase_1, 'R'), probabilidadeCD(frase_1, 'I')

In [525]:
# Comparativo: 
acertos = 0
for i in range(len(test) - 1):
    frase_1 = list(tokenize(test['Clean'][i]))
    probFraseDadoR, probFraseDadoI = probabilidadeCD(frase_1, 'R'), probabilidadeCD(frase_1, 'I')
    probRdadoFrase = probFraseDadoR * P_R
    probIdadoFrase = probFraseDadoI * P_Rc
    if probRdadoFrase > probIdadoFrase:
        relevancia_prevista = 1
    else:
        relevancia_prevista = 0
    if relevancia_prevista == test['Relevancia'][i]:
        acertos += 1
    #print(i, relevancia_prevista, test['Relevancia'][i])
print('percentual de acerto: ', acertos / len(test))

percentual de acerto:  0.565


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**